# Creating a new audio track:

In [6]:
import reapy

def create_audio_track(project=None, index=None, track_name="RPR_AddMediaItemToTrack(MediaTrack tr)"):
    """
    Description: This function creates a new audio track at a required position.
                 Position 1 is index 0, Position 2 is index 1...
                 The default position is -1, which is at the end of the track list.

    Input Parameters:
    project (reapy.Project) - The opened project (optional, if None provided, a new project will be created).
    index (int) - The index at which you want to add the track (optional, default is at the end of all tracks).
    track_name (str) - The name of the new track you want to add (optional, default is "Track").

    Return Parameter:
    int - The ID of the new track.
    """

    if project is None:
        project = reapy.Project()
        
    if track_name == "RPR_AddMediaItemToTrack(MediaTrack tr)":
        track_name += f" {index}"
    
    new_track = project.add_track(index)
    new_track.name = track_name
    
    print(f"A new track has been created at index {index} and named {track_name}")
    
    return new_track.id


# Creating a New Project:

In [7]:
import reapy

def create_project():
    """
    Description: Create a project in Reaper using the reapy library.

    Input Parameters: None

    Return Parameters:
    reapy.Project - The created project.
    """
    project = reapy.Project()
    return project


# Muting a track:

In [8]:
import reapy

def mute_track(track_index=None, track_name='', opp=1, all=0):
    """
    Description: This function mutes, unmutes, or toggles the mute button of a track.

    Input Parameters:
    track_index (int) - The position of the track (0 for the first track, 1 for the second, etc.)
    track_name (str) - The name of the track, in case you want to select it by its name.
    opp (int) - opp = 1: Mutes the track
                opp = -1: Toggles the Mute Button (If track is muted, it unmutes it, and vice versa)
                opp = 0: Unmutes the track
    all (int) - all = 0: The action is done for only the specified track
                all = 1: The action is done for all tracks
                all = 2: The action is undone for all tracks

    Return Parameter: None
    """
    project = reapy.Project()
    track_id = None

    if track_name != '':
        for track in project.tracks:
            if track.name == track_name:
                track_id = track.id
                break
    else:
        track_id = project.tracks[track_index].id

    if all == 1:
        reapy.MuteAllTracks(opp)
    elif all == 2:
        for track in project.tracks:
            reapy.SetTrackUIMute(track.id, 0, 1)
    else:
        reapy.SetTrackUIMute(track_id, opp, 1)


# Adding Fade In to a track:

In [9]:
import reapy

def add_fade_in_to_media_item(track_index=0, item_index=0, fade_in_length=0.5, fade_in_shape=0):
    """
    Description: This function adds a fade-in effect to a media item on a specific track.

    Input Parameters:
    track_index (int) - The index of the track where the media item is located.
    item_index (int) - The index of the media item within the track.
    fade_in_length (float) - The length of the fade-in effect in seconds.
    fade_in_shape (float) - The shape of the fade-in curve (0 for linear, 1 for logarithmic, etc.).

    Return Parameter: None
    """
    project = reapy.Project()
    track = project.tracks[track_index]

    if item_index < 0 or item_index >= len(track.items):
        print("Invalid item index.")
        return

    media_item = track.items[item_index]
    start_position = media_item.position
    fade_in_position = start_position + fade_in_length

    media_item.set_fade_in(fade_in_length, fade_in_shape)

    print(f"Fade-in effect of length {fade_in_length} seconds and shape {fade_in_shape} added to media item at index {item_index} in track at index {track_index}.")


# Adjusting Master Volume and Pan:

In [10]:
import reapy

def adjust_master_volume_pan(vol_pan=0, crement_flag=0, set_value=3):
    """
    Description: This function alters the volume or pan of the master track.

    Input Parameters:
    vol_pan (int) - vol_pan = 0: Alter the volume
                  - vol_pan = 1: Alter the pan
    crement_flag (int) - crement_flag = 0: Set the pan/volume to a specified value (set_value).
                       - crement_flag = 1: Increase the pan/volume by set_value amount.
                       - crement_flag = 2: Change the pan/volume to a fraction (set_value) of its current value.
    set_value (float) - The extent to which the pan/volume must be changed.

    Return Parameter: None
    """
    project = reapy.Project()

    if vol_pan == 0:
        current_volume = project.master_track.volume
        if crement_flag == 0:
            new_value = set_value
        elif crement_flag == 1:
            new_value = current_volume + set_value
        else:
            new_value = current_volume * set_value

        project.master_track.volume = new_value
        print(f"The volume of the master track has been changed to {new_value} dB")

    elif vol_pan == 1:
        current_pan = project.master_track.pan
        if crement_flag == 0:
            new_value = set_value / 100
        elif crement_flag == 1:
            new_value = current_pan + set_value / 100
        else:
            new_value = current_pan * set_value / 100

        project.master_track.pan = new_value
        print(f"The pan has been changed to {new_value * 100}%")


# Moving the playhead forward and backward or fast forward/rewind.

In [14]:
def move_playhead(seconds: float):
    '''
    Description:
    This function moves the playhead forward or backward by a specified number of seconds.

    Input Parameters:
    seconds (float): The number of seconds to move the playhead. Positive value moves forward, negative value moves backward.

    Return Parameter:
    None
    '''
    RPR_CSurf_OnPlay()

    # Get the current play position in seconds
    current_pos = RPR.GetPlayPosition()

    # Calculate the new position after moving
    new_pos = current_pos + seconds

    # Move the playhead to the new position
    RPR.CSurf_OnSeek(new_pos, False)

    # Update the display and reset the play state
    RPR.UpdateArrange()
    RPR_CSurf_OnStop()
    print(f"Playhead moved {seconds} seconds {'forward' if seconds >= 0 else 'backward'}.")


# Duplicate track

In [15]:
def duplicate_track(project: reapy.Project, track_index: int, new_track_name: str = None):
    '''
    Description:
    This function duplicates an existing track in the project.

    Input Parameters:
    project (reapy.Project): The Reaper project object.
    track_index (int): The index of the track to be duplicated.
    new_track_name (str): Optional. The name of the new duplicated track. If not provided, it will be named "TrackX" where X is the next available number.

    Return Parameter:
    The ID of the newly duplicated track.
    '''
    track_to_duplicate = project.tracks[track_index]
    new_track = track_to_duplicate.duplicate()
    
    if not new_track_name:
        new_track_name = f"Track{len(project.tracks)}"
    
    # Set the name of the new track
    new_track.name = new_track_name
    
    print(f"Track {track_to_duplicate.name} duplicated as {new_track_name}.")
    
    return new_track.id


# Apply ReaGate to separate loud notes

In [16]:
def apply_reagate(track: reapy.Track, threshold_db: float = -30, attack_ms: float = 10, hold_ms: float = 50, release_ms: float = 10):
    '''
    Description:
    This function applies ReaGate to a track to separate loud notes.

    Input Parameters:
    track (reapy.Track): The Reaper track object to which ReaGate will be applied.
    threshold_db (float): Optional. The gate threshold in decibels. Default is -30 dB.
    attack_ms (float): Optional. The attack time in milliseconds. Default is 10 ms.
    hold_ms (float): Optional. The hold time in milliseconds. Default is 50 ms.
    release_ms (float): Optional. The release time in milliseconds. Default is 10 ms.

    Return Parameter:
    None
    '''
    plugin_name = "ReaGate"
    fx = track.add_fx(plugin_name)
    
    # Set ReaGate parameters
    fx.set_param("Threshold", threshold_db)
    fx.set_param("Attack", attack_ms)
    fx.set_param("Hold", hold_ms)
    fx.set_param("Release", release_ms)
    
    print(f"{plugin_name} applied to {track.name} with Threshold: {threshold_db} dB, Attack: {attack_ms} ms, Hold: {hold_ms} ms, Release: {release_ms} ms.")
